## 

## Before starting - setup index

This got NDCG ~0.29

```
cd notebooks/elasticsearch/vmware
python extract_use_es.py 0    # base reindex step
python extract_use_es.py 1    # enrich with USE
python extract_use_es.py 2    # extract first line from raw text
python extract_use_es.py 3    # extract remaining lines from raw text
```

In [1]:
from ltr.client import ElasticClient
client=ElasticClient()
vmware_corpus = client.to_dataframe(index='vmware')

Scanned 0 documents -- 0.506444013999996
Scanned 10000 documents -- 5.606228834999996
Scanned 20000 documents -- 11.212749281
Scanned 30000 documents -- 16.396668858999995
Scanned 40000 documents -- 21.513799333999998
Scanned 50000 documents -- 26.909003130000002
Scanned 60000 documents -- 32.034268134
Scanned 70000 documents -- 37.522221013999996
Scanned 80000 documents -- 43.100742959
Scanned 90000 documents -- 48.178119128999995
Scanned 100000 documents -- 54.430766379000005
Scanned 110000 documents -- 59.86330767999999
Scanned 120000 documents -- 65.940515848
Scanned 130000 documents -- 70.989507464
Scanned 140000 documents -- 75.898333947
Scanned 150000 documents -- 82.09155465799999
Scanned 160000 documents -- 87.169646501
Scanned 170000 documents -- 92.263125279
Scanned 180000 documents -- 98.88383648399999
Scanned 190000 documents -- 104.161401411
Scanned 200000 documents -- 109.34560248999999
Scanned 210000 documents -- 114.76016189900001
Scanned 220000 documents -- 121.601551

In [68]:
vmware_corpus.columns

Index(['type', 'subject', 'title', 'abstract', 'description', 'keywords',
       'guid', 'format', 'language', 'last-modified', 'author', 'product',
       'id', 'f_name', 'raw_text', 'document_group', 'enrich_version',
       'raw_text_use', 'first_line', 'remaining_lines', 'machine-translated',
       'edit-review-id', 'name', 'custom_label', 'link_url', 'children', 'url',
       'lastModified', 'md5', 'microsites-utag', 'onetrust-data-domain',
       'viewport', 'titleTag', 'robots', 'twitter:card', 'twitter:description',
       'twitter:title', 'twitter:image', 'generator', 'twitter:site',
       'twitter:creator', 'msapplication-TileImage', 'referrer',
       'uberflip:hub_id', 'application-name', 'msapplication-starturl',
       'msapplication-TileColor', 'google-site-verification',
       'twitter:partner', 'aem-utag', 'aem-cn-utag', 'date',
       'baidu-site-verification', 'product_versions', 'views', 'published',
       'modified', 'created', 'microsites-at-utag', 'Generator'

In [ ]:
type = 'concept'
for doc in vmware_corpus[vmware_corpus['type'] == 'task'].sample(frac=0.1).head().to_dict(orient='records'):
    print(doc['title'])
    print('--------')
    print(doc['first_line'])
    print('--------')
    print(doc['remaining_lines'])
    print('*********')

In [7]:
from ltr.client import ElasticClient
client=ElasticClient()

import pandas as pd
queries = pd.read_csv("data/test.csv")

def search(client, query):
    es = client.es
    body = {
        'size': 5,
        'query': {
            'bool': { 'should': [
                {'match_phrase': {
                    'remaining_lines': {
                        'slop': 10,
                        'query': query
                    }
                }},
                {'match_phrase': {
                    'first_line': {
                        'slop': 10,
                        'query': query
                    }
                }},
                {'match': {
                    'raw_text': {
                        'query': query
                    }
                }},
                {'match': {
                    'first_line': {
                        'query': query
                    }
                }},
            ]}
        }
    }

    return es.search(index='vmware', body=body)['hits']['hits']


def search_all(client, queries, how=search, verbose=False):
    all_results = []
    for query in queries.to_dict(orient='records'):
        results = how(client, query['Query'])
        for rank, result in enumerate(results):
            source = result['_source']
            if verbose and rank == 0:
                print(f"First result for for {query['QueryId']},{query['Query']}")
                if 'titleTag' in source:
                    print(source['titleTag'])
                else:
                    print("No title tag")
                if 'first_line' in source:
                    print(source['first_line'])
                else:
                    print("No first_line")
                    
                if 'remaining_lines' in source:
                    print(source['remaining_lines'])
                    
            source['rank'] = rank
            source['score'] = result['_score']
            source['DocumentId'] = source['id']
            source['QueryId'] = query['QueryId']
            all_results.append(source)
    all_results = pd.DataFrame(all_results)
    return queries.merge(all_results, how='left', on='QueryId').sort_values(['QueryId', 'rank'])
    
submission = search_all(client, queries)
submission

,QueryId,Query,viewport,microsites-at-utag,onetrust-data-domain,titleTag,title,twitter:card,description,datePublished,...,application-name,msapplication-starturl,msapplication-TileColor,name,custom_label,link_url,children,url,lastModified,md5
0,0,what is hypervisor,"width=device-width, initial-scale=1.0",https://tags.tiqcdn.com/utag/vmware/microsites...,b9242434-9feb-47be-a894-3a9e658fdd50,What is a Hypervisor? | VMware Glossary,What is a Hypervisor? | VMware Glossary,summary,"Hypervisor, also known as a virtual machine mo...",11-09-2020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,what is hypervisor,"width=device-width, initial-scale=1, minimum-...",NaN,NaN,What is a vSphere Hypervisor? | Free Hyperviso...,What is a vSphere Hypervisor? | Free Hyperviso...,summary_large_image,vSphere Hypervisor is a bare-metal hypervisor ...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,what is hypervisor,"width=device-width, initial-scale=1.0",https://tags.tiqcdn.com/utag/vmware/microsites...,b9242434-9feb-47be-a894-3a9e658fdd50,What is a Cloud Hypervisor? | VMware Glossary,What is a Cloud Hypervisor? | VMware Glossary,summary,A Cloud Hypervisor is software that enables th...,02-19-2021,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,what is hypervisor,"width=device-width, initial-scale=1.0",https://tags.tiqcdn.com/utag/vmware/microsites...,b9242434-9feb-47be-a894-3a9e658fdd50,What is a Bare Metal Hypervisor? | VMware Glos...,What is a Bare Metal Hypervisor? | VMware Glos...,summary,The bare metal hypervisor is the most commonly...,12-21-2020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,what is hypervisor,"width=device-width, initial-scale=1, minimum-...",NaN,NaN,What is ESXI? | Bare Metal Hypervisor | ESX | ...,What is ESXI? | Bare Metal Hypervisor | ESX | ...,summary,VMware ESXi formerly known as ESX is a bare me...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11750,2350,kubernetes what is a namespace,"width=device-width, initial-scale=1.0",https://tags.tiqcdn.com/utag/vmware/microsites...,b9242434-9feb-47be-a894-3a9e658fdd50,What is a Kubernetes Namespace? | VMware Glossary,What is a Kubernetes Namespace? | VMware Glossary,summary,Namespaces are a way to organize clusters into...,11-18-2020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11751,2350,kubernetes what is a namespace,NaN,NaN,NaN,NaN,Configure a Kubernetes Zone in Cloud Assembly,NaN,Kubernetes zones enable cloud administrators t...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11752,2350,kubernetes what is a namespace,NaN,NaN,NaN,NaN,Configure a Kubernetes Zone in vRealize Automa...,NaN,Kubernetes zones enable cloud administrators t...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11753,2350,kubernetes what is a namespace,NaN,NaN,NaN,NaN,Configure a Kubernetes Zone in vRealize Automa...,NaN,Kubernetes zones enable cloud administrators t...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
def write_submission(name='pull_out_firstline'):
    from time import time
    timestamp = str(time()).replace('.', '')
    fname = f'data/{name}_turnbull_{timestamp}.csv'
    print("Writing To: ", fname)
    submission[['QueryId', 'DocumentId']].to_csv(fname, index=False)

In [9]:
write_submission()

Writing To:  data/pull_out_firstline_turnbull_1653265242749123.csv


In [10]:
!ls data/

pull_out_firstline_turnbull_1653253060074837.csv
pull_out_firstline_turnbull_1653265242749123.csv
sample_submission.csv
test.csv
turnbull_submission_1652544485247169.csv
turnbull_submission_1652544652859947.csv
turnbull_submission_1652544680901428.csv
use_feedback_rrf_turnbull_submission_1653226391886872.csv
vmware_ir_content.csv
vmware_ir_content_parsed.pkl


In [14]:
regular_bm25 = pd.read_csv('data/turnbull_submission_1652544680901428.csv')
with_firstline_bm25 = pd.read_csv('data/pull_out_firstline_turnbull_1653253060074837.csv')

In [37]:
def group_as_list(group):
    return group['DocumentId'].tolist()


def jaccard(row):
    before = set(row['before'])
    after = set(row['after'])
          
    return len(before & after) / len(before | after)


diff = regular_bm25.groupby('QueryId').apply(group_as_list).to_frame().rename(columns={0: 'before'})
diff['after'] = with_firstline_bm25.groupby('QueryId').apply(group_as_list)

diff['jaccard'] = diff.apply(jaccard, axis=1)

In [61]:
diff[diff['jaccard'] < 1.0].sort_values('jaccard')

,before,after,jaccard
QueryId,,,
1173,"[horizon-html-access-2103-release-notes.txt, h...",[en-2021-VMware-Horizon-HTML-Access-4-6-com-vm...,0.000000
1555,[https---www.vmware.com-solutions-windows10-mo...,[https---techzone.vmware.com-blog-how-provisio...,0.000000
1552,[en-2021-VMware-Fusion-8-0-com-vmware-fusion-u...,[https---blogs.vmware.com-vmtn-2009-05-how-to-...,0.000000
638,[https---blogs.vmware.com-cloudprovider-2020-0...,[https---blogs.vmware.com-management-2017-04-w...,0.000000
1550,[en-2021-VMware-Skyline-1-4-user-guide-GUID-78...,"[80791.txt, 2103054.txt, 55609.txt, 56438.txt,...",0.000000
...,...,...,...
520,[en-2021-vSphere-Replication-8-4-com-vmware-vs...,[en-2021-VMware-vSphere-5-5-com-vmware-vsphere...,0.666667
1713,"[2126729.txt, 2126733.txt, 2057907.txt, 212861...","[2126733.txt, 2126729.txt, 2128613.txt, 101841...",0.666667
1706,"[https---tanzu.vmware.com-event-streaming.txt,...","[https---tanzu.vmware.com-event-streaming.txt,...",0.666667


In [91]:
queries[queries['QueryId'] == 1552]

,QueryId,Query
1552,1552,how to use vmware fusion on mac


In [92]:
def rebuild_query_results(query_id, results):
    return results[results['QueryId'] == query_id].merge(vmware_corpus, 
                                                         left_on='DocumentId',
                                                         right_on='f_name', how='left')

In [94]:
for result in rebuild_query_results(1552, regular_bm25).to_dict(orient='record'):
    print(result['first_line'])
    print(result['remaining_lines'][:15])
    print('--------')

With Fusion, you can run personal computer (PC) applications and devices on your Intel-based Mac.
['Designed for the Mac user, Fusion takes advantage of the security, flexibility, and portability of virtual machines to run Windows and other x86 operating systems at the same time as Mac OS X.', 'About VMware Fusion', 'Take a quick look at what Fusion does and how it works.', 'About VMware Fusion Pro', 'Fusion Pro provides advanced virtualization features for managing the virtual machines on your Mac.', 'System Requirements for Fusion', 'Verify that you have the following minimum system requirements for installing and using Fusion.', 'install Fusion', 'You install Fusion in the same way you install other Mac OS X applications.', 'Start Fusion', 'Start using Fusion.', 'How-to Videos', 'Watch online videos about using Fusion.', 'Take Advantage of Fusion Online Resources', 'Go to the VMware Fusion home page and support page for product news, FAQs, more resources, and the online community.']

<ipython-input-94-8c27dda90f2f>:1: FutureWarning:

Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.



In [95]:
for result in rebuild_query_results(1552, with_firstline_bm25).to_dict(orient='record'):
    print(result['first_line'])
    print(result['remaining_lines'][:5])
    print('--------')

How to Get Windows 7 on Mac with VMware Fusion - VMTN Blog
['Share on:Share on Twitter', 'Share on LinkedIn', 'Share on Facebook', 'Email this post', '0']
--------
Fusion - Run Windows on Mac | VM for Mac | VMware
['<script type="application/ld+json">', '[{', '"@context": "http://schema.org/",', '"@type": "Product",', '"name": "Fusion",']
--------
Ars Technica Reviews Linux GIMP on Mac using VMware Fusion - VMware Fusion Blog - VMware Blogs
['Home > Blogs > VMware Fusion Blog', 'VMware Fusion Blog', 'Insight and highlights from the VMware Fusion Team for running Windows on your Mac', 'Post navigation', '← VMware Fusion at Macworld Video Interview with TUAWMicrosoft Songsmith on the Mac with VMware Fusion →..']
--------
Twitter Relies on the Mac and VMware Fusion - VMware Fusion Blog - VMware Blogs
['Home > Blogs > VMware Fusion Blog', 'VMware Fusion Blog', 'Insight and highlights from the VMware Fusion Team for running Windows on your Mac', 'Post navigation', '← Ubuntu 9.04 "Jaunty Jac

<ipython-input-95-fee4939e8fbc>:1: FutureWarning:

Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.



In [52]:
for resp in search(client, 'what is a virtual machine image'):
    print(resp['_source']['first_line'])

What is a Virtual Machine? | VMware Glossary
Connection Server requires a golden image virtual machine from which you generate a base image for creating instant clones.
Connection Server requires a golden image virtual machine from which you generate a base image for creating instant clones.
Connection Server requires a golden image virtual machine from which you generate a base image for creating instant clones.
What is virtualization technology & virtual machine? | VMware
